### 1. Read data
Read the data from CSV and apply some basic pre-processing (remove non-ascii characters, convert our target variable to an integer label).

In [1]:
import numpy as np
import pandas as pd
from nltk import word_tokenize
from imblearn.under_sampling import RandomUnderSampler

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
path = r"../input/train.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df.shape

(1306122, 3)

In [5]:
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
docs = df["question_text"].values
labels = df["target"].values

In [7]:
np.unique(labels,return_counts=True)

(array([0, 1]), array([1225312,   80810]))

### 2. Preprocessing
Tokenize text, convert words / tokens to indexed integers. Take each document and convert to a sequence of max length 20 (pad with zeroes if shorter).

In [8]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(vocab_size)

222162


In [9]:
#t.word_index

In [10]:
# encoded_docs

In [11]:
# text_lens=[]
# for title in docs:
#     text_lens.append(len(word_tokenize(title)))

In [12]:
# max(text_lens)

In [13]:
# np.quantile(text_lens,0.85)

In [14]:
# pad documents to a max length of 4 words
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

1306122


In [15]:
padded_docs.shape

(1306122, 20)

In [16]:
rus = RandomUnderSampler(random_state=42)
padded_docs_rus,labels_rus = rus.fit_resample(padded_docs,labels)

In [17]:
padded_docs_rus.shape,labels_rus.shape,padded_docs.shape,labels.shape

((161620, 20), (161620,), (1306122, 20), (1306122,))

In [18]:
np.unique(labels_rus,return_counts=True)

(array([0, 1]), array([80810, 80810]))

### 3. Import embeddings
The clever part: import a dictionary of word embeddings that translates each word into a 100 dimensional vector.

In [19]:
# load the whole embedding into memory
EMBEDDING_FILE = r"../input/glove.840B.300d.txt"
EMBEDDING_DIM = 100

def get_embedding():
    embeddings_index = {}
    f = open(EMBEDDING_FILE,'r', errors = 'ignore', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
        except:
            pass
            
    f.close()
    return embeddings_index

embeddings_index = get_embedding()

In [20]:
# create a weight matrix for words in training docs

def create_embedding_weights(vocab_size,t):
    embedding_matrix = np.zeros((vocab_size, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print(embedding_matrix.shape)
    return embedding_matrix



In [21]:
embedding_matrix = create_embedding_weights(vocab_size,t)

(222162, 300)


### 4. Network architecture

##### Simple model

In [22]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=20, trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           66648600  
_________________________________________________________________
flatten_1 (Flatten)          (None, 6000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6001      
Total params: 66,654,601
Trainable params: 6,001
Non-trainable params: 66,648,600
_________________________________________________________________
None


##### LSTM model

In [23]:
## create model
model_glove = Sequential()
model_glove.add(Embedding(vocab_size, 300, input_length=20, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model_glove.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 300)           66648600  
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 300)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 16, 64)            96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 66,810,765
Trainable params: 162,165
Non-trainable params: 66,648,600
__________________________________________________________

### 5. Training and Evaluation
Is it any good? Let's find out.
Divide our dataset using a holdout strategy:

In [25]:
# split dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs_rus, labels_rus, test_size=0.2, random_state=42)

In [26]:
# fit the model
model_glove.fit(X_train, y_train, epochs=5, verbose=0)

In [27]:
# evaluate the model
loss, accuracy = model_glove.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 88.513179


In [28]:
model_glove.predict(X_test)

array([[0.5927925 ],
       [0.9738952 ],
       [0.01817387],
       ...,
       [0.06050098],
       [0.9102117 ],
       [0.97154105]], dtype=float32)

In [29]:
ques = ['Has the United States become the largest dictatorship in the world?','How should I prepare for IIT K/IIM C/ ISI K PGDBA course exam and interview?']

In [30]:
t.fit_on_texts(ques)
# integer encode the documents
encoded_ques = t.texts_to_sequences(ques)
max_length = 20
padded_ques = pad_sequences(encoded_ques, maxlen=max_length, padding='post')
print(len(padded_ques))

2


In [31]:
model_glove.predict(padded_ques)

array([[0.56878805],
       [0.01305632]], dtype=float32)

simple model evaluation

In [32]:
# fit the model
model.fit(X_train, y_train, epochs=5, verbose=0)

In [33]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 86.025863


In [34]:
model.predict(X_test)

array([[0.11535314],
       [0.99393773],
       [0.39336076],
       ...,
       [0.7290954 ],
       [0.8021682 ],
       [0.99986327]], dtype=float32)

In [35]:
ques = ['Has the United States become the largest dictatorship in the world?','How should I prepare for IIT K/IIM C/ ISI K PGDBA course exam and interview?']

In [36]:
t.fit_on_texts(ques)
# integer encode the documents
encoded_ques = t.texts_to_sequences(ques)
max_length = 20
padded_ques = pad_sequences(encoded_ques, maxlen=max_length, padding='post')
print(len(padded_ques))

2


In [37]:
model.predict(padded_ques)

array([[0.35528618],
       [0.01015047]], dtype=float32)